In [13]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv()
IMAGES_PATH = os.getenv('IMAGES_PATH')
ROOT_DATA_PATH = os.getenv('ROOT_DATA_PATH')
BATCH_SIZE= int(os.getenv('BATCH_SIZE'))
EPOCHS = int(os.getenv('EPOCHS'))
LR = float(os.getenv('LR'))
PATCH_SIZE=int(os.getenv('PATCH_SIZE'))
DROPOUT=float(os.getenv('DROPOUT'))
ATTENTION_DROPOUT=float(os.getenv('ATTENTION_DROPOUT'))
HIDDEN_DIM=int(os.getenv('HIDDEN_DIM'))
MLP_DIM=int(os.getenv('MLP_DIM'))
NUM_HEADS=int(os.getenv('NUM_HEADS'))
NUM_LAYERS=int(os.getenv('NUM_LAYERS'))


In [7]:
import torch
from torchvision.models import VisionTransformer
from utilities import train_step, test_step

model = VisionTransformer(image_size=80, patch_size=PATCH_SIZE, num_layers=NUM_LAYERS, 
                                       num_heads=NUM_HEADS, hidden_dim=HIDDEN_DIM, 
                                       mlp_dim=MLP_DIM, dropout=DROPOUT,
                                       attention_dropout=ATTENTION_DROPOUT,
                                       num_classes=2)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
from preprocess.data_process import get_dataloaders
from torch.utils.data import DataLoader

if os.path.exists('train_set.pt'):
    train_set = torch.load('train_set.pt')
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
if os.path.exists('val_set.pt'):
    val_set = torch.load('val_set.pt')
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
if os.path.exists('test_set.pt'):
    test_set = torch.load('test_set.pt')
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)
        
else:
    train_loader, val_loader, test_loader = get_dataloaders()

In [ ]:
from timeit import default_timer as timer
from tqdm import tqdm
import pandas as pd
train_start_time = timer()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on Device: {device}")
model.to(device)

# Create two empty dataframes to store loss and accuracy values
train_df = pd.DataFrame(columns=['Loss', 'Accuracy'])
test_df = pd.DataFrame(columns=['Loss', 'Accuracy'])

print("-----------Training starting----------")
for epoch in tqdm(range(EPOCHS)):
    if epoch % 10 == 0:
        print(f"Epoch {epoch + 1}/{EPOCHS}")
    
    # Train the model
    train_loss, train_acc = train_step(model, loss_fn, optimizer, device, train_loader)
    test_loss, test_acc = test_step(model, loss_fn, device, test_loader)

    # Accumulate the loss and accuracy for both train and test after each epoch
    train_df.append({'Loss': train_loss.item(), 'Accuracy': train_acc}, ignore_index=False)
    test_df.append({'Loss': test_loss.item(), 'Accuracy': test_acc}, ignore_index=False)

print("-----------Training finished----------")

train_end_time = timer()

total_train_time = train_end_time - train_start_time

print(f"Training completed in {total_train_time:.2f} seconds")